# Olist Customer Segmentation & High-Value Customer Prediction (RFM)

**Goal:** Turn raw e-commerce transactions into customer segments and an interpretable model that identifies high-value customers.

This notebook follows a narrative format:
**Question → Method → Output → Interpretation → Next step.**


## Step 1 — Build an analysis-ready dataset (Order-level)

**Why this step exists:**  
Raw Olist data is split across multiple tables (orders, payments, customers).  
Before analysis, we need a clean **order-level table** with:
- delivered orders only (consistent “completed” transactions)
- total payment value per order (monetary base)
- timestamps needed for downstream customer features


In [1]:
# Import Libraries
import pandas as pd
import numpy as np

#Load Raw Data
orders = pd.read_csv("olist_orders_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")

### 1.1 Filter to delivered orders

**Question:** Which transactions represent completed customer experience?  
**Decision:** Use `delivered` orders to avoid mixing cancellations with completed purchases.

This ensures monetary and recency metrics reflect actual delivered transactions.


In [2]:
#Filter Delivered Orders Only
orders_delivered = orders[orders["order_status"]=="delivered"].copy()

### 1.2 Aggregate payments to order level

**Problem:** One order may have multiple payment records (installments / split payments).  
**Goal:** Create a single `order_payment_total` per order to measure revenue consistently.

This step prevents double-counting when we later join payments into orders.


In [3]:
#Aggregate Payments at Order Level
payment_per_order = (
    payments.groupby("order_id", as_index = False).agg({"payment_value": "sum"}) #revenue
)

### 1.3 Join orders with order-level payments

Now we combine:
- order timestamps/status (from orders)
- total payment value (from aggregated payments)

This produces a unified order-level dataset for customer feature engineering.


In [4]:
#Join Orders with Payments
orders_with_payment = orders_delivered.merge(
    payment_per_order,
    on = "order_id",
    how = "left"
)

#Attatch Customer Identifier
order_level = orders_with_payment.merge(
    customers[["customer_id", "customer_unique_id"]],
    on = "customer_id",
    how = "left"
)

In [5]:
#Select Relevant Columns
order_level = order_level[[
    "customer_unique_id",
    "order_id",
    "order_purchase_timestamp",
    "payment_value"
]]

#Convert Timestamp to Datetime
order_level["order_purchase_timestamp"] = pd.to_datetime(
    order_level["order_purchase_timestamp"]
)

### 1.4 Validate the order-level dataset

Before moving to customer analytics, we validate:
- row counts
- uniqueness of `order_id`
- missing timestamps

This prevents subtle aggregation/join mistakes from contaminating downstream results.


In [6]:
#Final Validation
print(order_level.shape)
order_level.head()

(96478, 4)


,customer_unique_id,order_id,order_purchase_timestamp,payment_value
0,7c396fd4830fd04220f754e42b4e5bff,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,38.71
1,af07308b275d755c9edb36a90c618231,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,141.46
2,3a653a41f6f9fc3d2a113cf8398680e8,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,179.12
3,7c142cf63193a1473d2e66489a9ae977,949d5b44dbf5de918fe9c16f97b45f8a,2017-11-18 19:28:06,72.20
4,72632f0f9dd73dfee390c9b22eb56dd6,ad21c59c0840e6cb83a9ceb5573f8159,2018-02-13 21:18:39,28.62


## Step 2 — Create customer-level RFM features

**Why RFM:**  
To compare customers, we need consistent behavioral metrics:
- **Recency:** how recently they purchased
- **Frequency:** how many orders they placed
- **Monetary:** how much they spent

We define a reference date (the day after the latest purchase) to calculate recency in days.


In [7]:
ref_date = order_level["order_purchase_timestamp"].max().normalize()+pd.Timedelta(days = 1)
print("Reference date: ",ref_date)

Reference date:  2018-08-30 00:00:00


### 2.1 Aggregate orders into customer behavior

**Plan:**  
For each `customer_unique_id`, compute:
- last purchase date → recency
- number of distinct orders → frequency
- total spend → monetary

This transforms order transactions into a customer analytics table.


In [8]:
customer_features = (
    order_level
    .groupby("customer_unique_id", as_index = False)
    .agg(
        last_purchase = ("order_purchase_timestamp", "max"),
        frequency = ("order_id", "nunique"),
        monetary = ("payment_value","sum")
    )
)
customer_features.head()

,customer_unique_id,last_purchase,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89


In [9]:
customer_features["recency_days"] = (ref_date - customer_features["last_purchase"]).dt.days

In [10]:
customer_features = customer_features[[
    "customer_unique_id",
    "recency_days",
    "frequency",
    "monetary"
]]
customer_features.head()

,customer_unique_id,recency_days,frequency,monetary
0,0000366f3b9a7992bf8c76cfdf3221e2,111,1,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,1,27.19
2,0000f46a3911fa3c0805444483337064,537,1,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,321,1,43.62
4,0004aac84e0df4da2b147fca70cf8255,288,1,196.89


## Step 3 — Understand the customer base (RFM distribution)

Before clustering/modeling, we answer:
- Are most customers one-time buyers?
- How skewed is spending (monetary)?
- What does “typical” customer behavior look like?

We inspect summary statistics to decide whether segmentation is necessary.


In [11]:
print("Order-level rows:", len(order_level))
print("Distinct orders:",order_level["order_id"].nunique())
print("Customer rows:", len(customer_features))
print("Distinct customers:",customer_features["customer_unique_id"].nunique())

customer_features.describe()

Order-level rows: 96478
Distinct orders: 96478
Customer rows: 93358
Distinct customers: 93358


,recency_days,frequency,monetary
count,93358.000000,93358.000000,93358.000000
mean,237.478888,1.033420,165.197003
std,152.595050,0.209097,226.314012
min,0.000000,1.000000,0.000000
25%,114.000000,1.000000,63.052500
50%,218.000000,1.000000,107.780000
75%,346.000000,1.000000,182.557500
max,713.000000,15.000000,13664.080000


### Interpretation & next step

The distribution indicates customer value is highly imbalanced:
most customers purchase once, while a small group drives much higher spending.

**Next:** segment customers into distinct groups so we can compare behaviors and identify high-value segments.


## Step 4 — Segment customers (Clustering)

**Why clustering:**  
RFM shows customers are not homogeneous. Instead of using arbitrary thresholds,
we let the data reveal natural groupings.

**Important note:**  
RFM features have very different scales (e.g., monetary vs frequency).  
We standardize features so no single metric dominates distance-based clustering.


In [12]:
from sklearn.preprocessing import StandardScaler

In [13]:
scaler = StandardScaler()

rfm_scaled = scaler.fit_transform(
    customer_features[["recency_days", "frequency", "monetary"]]
)

In [14]:
from sklearn.cluster import KMeans

In [27]:
kmeans = KMeans(
    n_clusters = 4,
    random_state = 42,
    n_init = 10
)
customer_features["cluster"] = kmeans.fit_predict(rfm_scaled)
# customer_features.head()

In [16]:
cluster_summary = (
    customer_features
    .groupby("cluster")
    .agg(
        customers = ("customer_unique_id", "count"),
        avg_recency = ("recency_days", "mean"),
        avg_frequency = ("frequency", "mean"),
        avg_monetary = ("monetary", "mean")
    )
    .sort_values("avg_monetary", ascending = False)
)
cluster_summary

,customers,avg_recency,avg_frequency,avg_monetary
cluster,,,,
2,2417,238.987174,1.012826,1161.111837
3,2772,219.959235,2.114358,289.680253
0,50720,127.806309,1.000000,134.331119
1,37449,387.216187,1.000000,133.509211


## Insight → Action (Customer Segments)

### Insight
Clustering reveals distinct customer segments with different value profiles:

- **Cluster 2 (High Monetary Segment)**: very high spend (~$1,161 avg) but low frequency (~1), suggesting high-ticket purchases (e.g., expensive items).
- **Cluster 3 (Repeat Buyers)**: higher frequency (~2.1) with mid monetary (~$290), representing repeat engagement rather than big-ticket value.
- **Cluster 0 (Recent Low-Spend Majority)**: largest group with lower recency_days (~128) and low spend.
- **Cluster 1 (Dormant Low-Spend Majority)**: large group with very high recency_days (~387), likely churned or inactive.

This segmentation shows that “high value” can mean **high spend** or **repeat engagement**, and the business strategy should differ by segment.

### Action
A practical marketing / CX playbook by cluster:

- **Cluster 2 (High spend)**:  
  - VIP treatment, premium delivery options, fraud prevention, and concierge-style customer support  
  - “Protect the margin” because losing one customer means losing large revenue per order

- **Cluster 3 (Repeat buyers)**:  
  - Subscription-like loyalty offers, cross-sell bundles, and targeted personalized recommendations  
  - “Increase lifetime value” by encouraging recurring purchases

- **Cluster 1 (Dormant)**:  
  - Win-back campaigns, reactivation coupons, and “why you left” feedback collection  
  - “Recover lost demand” but control costs (don’t overspend on low-ROI segments)

### Next Step
Segmentation explains *what groups exist*. Next we translate the high-value segment into a binary label and build a model that can identify high-value customers early.


In [26]:
customer_features["cluster"].value_counts()

cluster
0    50720
1    37449
3     2772
2     2417
Name: count, dtype: int64

## Step 5 — Predict high-value customers (Classification)

Segmentation explains *what groups exist*, but businesses often need to predict
*who is likely to become high-value*.

**Plan:**
1) Choose the highest-value cluster as the “high-value” definition  
2) Create a binary label (`is_high_value`)  
3) Train an interpretable model (Logistic Regression) using RFM features  
4) Interpret coefficients as business signals


In [18]:
# Define high_value customers (Cluster 2)
customer_features["is_high_value"] = (
    customer_features["cluster"] == 2
).astype(int)

customer_features["is_high_value"].value_counts()

is_high_value
0    90941
1     2417
Name: count, dtype: int64

In [19]:
#RFM
x = customer_features[["recency_days", "frequency", "monetary"]]
y = customer_features["is_high_value"]

In [20]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x, y,
    test_size = 0.3,
    random_state = 42,
    stratify = y
)

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [22]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(
    class_weight = "balanced",
    random_state = 42,
    max_iter = 1000
)

log_reg.fit(x_train_scaled, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [23]:
from sklearn.metrics import classification_report

y_pred = log_reg.predict(x_test_scaled)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     27283
           1       0.84      1.00      0.92       725

    accuracy                           1.00     28008
   macro avg       0.92      1.00      0.96     28008
weighted avg       1.00      1.00      1.00     28008



### Interpretation

We focus on performance for the high-value class (minority group).  
In practice, **recall** matters because missing a high-value customer is costly.

**Next:** interpret coefficients to understand which behaviors drive high-value classification.


In [24]:
feature_importance = pd.DataFrame({
    "feature": x.columns,
    "coefficient": log_reg.coef_[0]
}).sort_values("coefficient", ascending = False)

feature_importance

,feature,coefficient
2,monetary,12.930172
0,recency_days,-0.403522
1,frequency,-8.382256


### Coefficient interpretation (business meaning)

- **Monetary (positive):** higher total spend strongly increases high-value probability.
- **Recency (negative):** larger recency_days means “less recent,” reducing high-value probability.
- **Frequency (negative, after controlling for monetary):** suggests that in this dataset,
  fewer high-value purchases are more indicative than frequent low-value purchases.
  
This is a realistic pattern when monetary and frequency are correlated and monetary dominates value.


## Final Takeaways

- Built an analysis-ready customer table from raw multi-table e-commerce transactions  
- Segmented customers using RFM-based clustering to reveal behavioral groups  
- Defined and predicted high-value customers with an interpretable classification model  
- Delivered actionable signals: monetary dominates value, recency matters, frequency adds nuance after controlling for spend

If applied in practice, this workflow supports targeted retention and prioritization strategies.
